In [23]:
import pandas as pd
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
import re
import os
import shutil
import sddk
import pickle
import json
import nltk
import pickle

In [48]:
metadata_table_long = pd.read_json("../data/metadata_table_long.json")
metadata_table_long.head(5)

,Author,Full title,In,Year,Place,Publisher/Printer,Era,Form/Genre,Discipline/Content,Original,...,ids,id,date_min,date_max,filename,file_year,sents_n,tokens_n,disciplines_list,science_class
0,"Achrelius, Daniel",Scientiarum magnes recitatus publice anno 1690...,None,1690,[Turku],Wall,17th century,Oration,"Mathematics, Astronomy/Astrology/Cosmography, ...",Scientiarum magnes(Google Books),...,[705665],705665,1690.0,1690.0,"Achrelius,_Daniel_-_Scientiarum_magnes__Turku_...",1690.0,630,8839,"[Mathematics, Astronomy/Astrology/Cosmography,...",mixture
1,"Acidalius, Valens","Ad Iordanum Brunum Nolanum, Italum","Poematum Iani Lernutii, Iani Gulielmi, Valenti...",1603,"Liegnitz, Wrocław","Albert, David",17th century,Panegyric poem,Astronomy/Astrology/Cosmography,Ad Iordanum Brunum (1603)(CAMENA)Ad Iordanum B...,...,[801745],801745,1603.0,1603.0,Janus_Lernutius_et_al__-_Poemata__Liegnitz_160...,1603.0,6527,82445,[Astronomy/Astrology/Cosmography],exact_sciences
2,"Acosta, José de",De natura novi orbis libri duo et De promulgat...,None,1589,Salamanca,Guillelum Foquel,16th century,Monograph,"Astronomy/Astrology/Cosmography, Geography/Car...",De natura novi orbis(Biodiversity Heritage Lib...,...,[713323],713323,1589.0,1589.0,"Acosta,_José_de_-_De_natura_novi_orbis__Salama...",1589.0,8905,170205,"[Astronomy/Astrology/Cosmography, Geography/Ca...",exact_sciences
3,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",None,1620,Heidelberg,"Rosa, Geyder",17th century,Biography,Medicine,Vitae Germanorum medicorum(MDZ)Alternative lin...,...,[693148],693148,1620.0,1620.0,"Adam,_Melchior_-_Vitae_Germanorum_medicorum__H...",1620.0,17961,193272,[Medicine],life_sciences
4,"Addison, Joseph",Ad insignissimum virum dominum Thomam Burnettu...,"Examen poeticum duplex, sive, Musarum anglican...",1698,London,Richard Wellington I.,17th century,Panegyric poem,Meteorology/Earth sciences,Ad Burnettum sacrae theoriae telluris auctorem...,...,[769230],769230,1698.0,1698.0,Examen_poeticum_duplex__London_1698_pdf.txt,1698.0,3533,47878,[Meteorology/Earth sciences],exact_sciences


In [25]:
filtered_vocab_df = pd.read_json("../data/filtered_vocab_df.json")
filtered_vocab_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,mean,in_lila_embeddings,in_lasla,in_operamaiora,transl
3299,dico,43255,122549,42457,72224,70121.25,True,True,True,"say, call, tell"
960,pars,33225,93059,44337,52385,55751.50,True,True,True,part
3871,possum,25029,82700,38391,76841,55740.25,True,True,True,"be able, can"
6039,habeo,31690,89199,39443,59770,55025.50,True,True,True,"have, hold, possess, consider, think"
2898,facio,31814,98524,36944,51438,54680.00,True,True,True,"do, make, handle"
5630,liber,39559,65900,38496,53100,49263.75,True,True,True,"book, volume, inner bark of a tree, book, book"
4259,res,29069,70669,28291,51565,44898.50,True,True,True,suddenly
1219,uideo,21536,59099,26372,50503,39377.50,True,True,True,
5787,locus,23102,56782,31421,43680,38746.25,True,True,True,"place, location"
1257,corpus,16979,63171,22871,31938,33739.75,True,True,True,"body, corpus"


In [26]:
filtered_vocab_df = filtered_vocab_df[filtered_vocab_df["word"].apply(lambda x: len(x) > 2)]

In [27]:
vocab_freqs = filtered_vocab_df.set_index("word")["1601-1650"].to_dict()

# FastText - development and testing

In [28]:
metadata_table_long["id"].tolist()[0]

705665

In [7]:
source_path = "/srv/data/tome/noscemus/sents_data_jsons/"
id = metadata_table_long["id"].tolist()[0]
f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
f_sents_data[10:15]

[['705665',
  10,
  'Adfulgeant Omnia Bona!',
  [['Adfulgeant', 'adfulgeo', 'VERB', [0, 10]],
   ['Omnia', 'omnis', 'DET', [11, 16]],
   ['Bona', 'Bonum', 'NOUN', [17, 21]],
   ['!', '!', 'PUNCT', [21, 22]]]],
 ['705665',
  11,
  'Digitized by Google Diuina Majestate * Signati Augustissime Augustissima Regum, Regina, Cogitatione, Consiliis, Affectu, Parentes Patriae!',
  [['Digitized', 'digitized', 'VERB', [0, 9]],
   ['by', 'by', 'SCONJ', [10, 12]],
   ['Google', 'Google', 'PROPN', [13, 19]],
   ['Diuina', 'diuinus', 'ADJ', [20, 26]],
   ['Majestate', 'maiestas', 'NOUN', [27, 36]],
   ['*', '*', 'PUNCT', [37, 38]],
   ['Signati', 'signatus', 'VERB', [39, 46]],
   ['Augustissime', 'augustissimus', 'ADV', [47, 59]],
   ['Augustissima', 'augustissimus', 'ADJ', [60, 72]],
   ['Regum', 'rex', 'NOUN', [73, 78]],
   [',', ',', 'PUNCT', [78, 79]],
   ['Regina', 'regina', 'NOUN', [80, 86]],
   [',', ',', 'PUNCT', [86, 87]],
   ['Cogitatione', 'cogitatio', 'NOUN', [88, 99]],
   [',', ',', 'PUNC

In [30]:
# a genrator for iteration without flooding the CPU
class SentsCorpus:
    def __iter__(self):
        source_path = "/srv/data/tome/noscemus/sents_data_jsons/"
        for id in ids:
            f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
            sents_n = len(f_sents_data)
            tokens_n = 0
            lemmata = []
            for (doc_id, sent_id, sent_text, sent_data) in f_sents_data:
                tokens_n += len(sent_data)
                lemmasent = []
                for wordform, lemma, tag, position in sent_data:
                    if tag in ["NOUN", "PROPN", "ADJ", "VERB"]:
                        lemmasent.append(lemma)
                lemmasent = [re.sub(r"\W*|\d*", "", t) for t in lemmasent]
                lemmasent = [l.lower() for l in lemmasent if len(l) > 2]
                try:
                    yield lemmasent
                except:
                    pass

In [18]:
# test with a small corpus from one decade
ids = metadata_table_long[metadata_table_long["file_year"].between(1601, 1610)]["id"]
vocab_freqs = filtered_vocab_df.set_index("word")["1601-1650"].to_dict()
corpus = SentsCorpus()

In [19]:
# how many sentences are there in the small corpus?
len([s for s in corpus])

194843

In [14]:
%%time
# let's train the model
model = FastText(vector_size=100, window=10, negative=25, ns_exponent=1, sg=1, epochs=15, workers=8, min_n=5)
model.build_vocab_from_freq(word_freq=vocab_freqs)
model.train(corpus, total_examples=len([s for s in corpus]), epochs=model.epochs)

CPU times: user 4min 3s, sys: 2.24 s, total: 4min 5s
Wall time: 1min 8s


(10942294, 16475280)

In [18]:
# most similar words to harmonia
model.wv.most_similar("harmonia")

[('concentus', 0.6198955178260803),
 ('inuiolabilis', 0.5702183246612549),
 ('conuenientus', 0.5379429459571838),
 ('compositus', 0.5328949689865112),
 ('compositio', 0.5327402353286743),
 ('accentus', 0.5318533182144165),
 ('concessio', 0.5280140042304993),
 ('incompositus', 0.5252606868743896),
 ('compono', 0.5224994421005249),
 ('structura', 0.5185607075691223)]

# Train the genre-based models

In [11]:
nltk.FreqDist([d for dislist in metadata_table_long[metadata_table_long["file_year"].between(1501,1700)]["disciplines_list"] for d in dislist]).most_common()

[('Medicine', 221),
 ('Astronomy/Astrology/Cosmography', 193),
 ('Biology', 141),
 ('Mathematics', 138),
 ('Meteorology/Earth sciences', 127),
 ('Physics', 105),
 ('Geography/Cartography', 84),
 ('Other (see description)', 79),
 ('Alchemy/Chemistry', 52)]

In [46]:
disciplines_list = ['Medicine',
 'Astronomy/Astrology/Cosmography',
 'Biology',
 'Mathematics',
 'Meteorology/Earth sciences',
 'Physics',
 'Geography/Cartography',
 'Alchemy/Chemistry']

In [39]:
total_vocabulary = list(filtered_vocab_df["word"])

In [44]:
discipline = "Biology"
ids = metadata_table_long[(metadata_table_long["file_year"].between(1501,1700)) & (metadata_table_long["disciplines_list"].apply(lambda x: discipline in x))]["id"]
corpus = SentsCorpus()
vocab_freqs_tups = nltk.FreqDist([l for sent in corpus for l in sent]).most_common()
vocab_freqs = dict([t for t in vocab_freqs_tups if t[0] in total_vocabulary])
vocab_freqs_tups[:10]

[('dico', 76957),
 ('liber', 65230),
 ('habeo', 61273),
 ('possum', 50255),
 ('uideo', 47212),
 ('facio', 42272),
 ('res', 42267),
 ('pars', 42118),
 ('genus', 40953),
 ('locus', 40768)]

In [43]:
len(vocab_freqs)

5938

In [22]:
model = FastText(vector_size=100, window=10, negative=25, ns_exponent=1, sg=1, epochs=15, workers=32, min_n=5)
model.build_vocab_from_freq(word_freq=vocab_freqs)
model.train(corpus, total_examples=len([s for s in corpus]), epochs=model.epochs)

NameError: name 'metadata_table_long' is not defined

In [47]:
%%time
# main training of the models
for discipline in disciplines_list:
    ids = metadata_table_long[(metadata_table_long["file_year"].between(1501,1700)) & (metadata_table_long["disciplines_list"].apply(lambda x: discipline in x))]["id"]
    corpus = SentsCorpus()
    vocab_freqs_tups = nltk.FreqDist([l for sent in corpus for l in sent]).most_common()
    vocab_freqs = dict([t for t in vocab_freqs_tups if t[0] in total_vocabulary])
    model = FastText(vector_size=100, window=10, negative=25, ns_exponent=1, sg=1, epochs=15, workers=32, min_n=5)
    model.build_vocab_from_freq(word_freq=vocab_freqs)
    model.train(corpus, total_examples=len([s for s in corpus]), epochs=model.epochs)
    discicipline_str = discipline.replace("/", "_").replace(" ", "_")
    print("model for the period {} successfully trained.".format(discicipline_str))
    model.wv.save("/srv/data/tome/noscemus/vectors/vectors_{}.wv".format(discicipline_str))

model for the period Medicine successfully trained.
model for the period Astronomy_Astrology_Cosmography successfully trained.
model for the period Biology successfully trained.
model for the period Mathematics successfully trained.
model for the period Meteorology_Earth_sciences successfully trained.
model for the period Physics successfully trained.
model for the period Geography_Cartography successfully trained.
model for the period Alchemy_Chemistry successfully trained.
CPU times: user 4h 54min 47s, sys: 1min 3s, total: 4h 55min 51s
Wall time: 2h 4min 8s


# FastText - training the temporal models

In [20]:
os.mkdir("/srv/data/tome/noscemus/vectors/")

In [23]:
periods = [(1501, 1550),
                      (1551, 1600),
                      (1601, 1650),
                      (1651, 1700)]
periods_str = filtered_vocab_df.columns[1:5]
periods_str

Index(['1501-1550', '1551-1600', '1601-1650', '1651-1700'], dtype='object')

In [ ]:
%%time
# main training of the models
for per_tup, per_str in zip(periods, periods_str):
    ids = metadata_table_long[metadata_table_long["file_year"].between(per_tup[0], per_tup[1])]["id"]
    vocab_freqs = filtered_vocab_df.set_index("word")[per_str].to_dict()
    corpus = SentsCorpus()
    model = FastText(vector_size=100, window=10, negative=25, ns_exponent=1, sg=1, epochs=15, workers=32, min_n=5)
    model.build_vocab_from_freq(word_freq=vocab_freqs)
    model.train(corpus, total_examples=len([s for s in corpus]), epochs=model.epochs)
    print("model for the period {} successfully trained.".format(per_str))
    model.wv.save("/srv/data/tome/noscemus/vectors/vectors_{}.wv".format(per_str))

In [23]:
# if you want to upload the vectors to sciencedata:
#shutil.make_archive("../data/large_data/vectors", 'zip', "../data/large_data/vectors") #
#s.s.put("https://sciencedata.dk/sharingout/kase%40zcu.cz/TOME/DATA/NOSCEMUS/vectors.zip", data=open("../data/large_data/vectors.zip", "rb"))
# shutil.make_archive("../data/large_data/lila_vectors", 'zip', "../data/large_data/lila_vectors") #
# s.s.put("https://sciencedata.dk/sharingout/kase%40zcu.cz/TOME/DATA/NOSCEMUS/lila_vectors.zip", data=open("../data/large_data/lila_vectors.zip", "rb"))

In [50]:
sorted([f for f in os.listdir("/srv/data/tome/noscemus/vectors/") if f.endswith(".wv")])

['vectors_1501-1550.wv',
 'vectors_1551-1600.wv',
 'vectors_1601-1650.wv',
 'vectors_1651-1700.wv',
 'vectors_Alchemy_Chemistry.wv',
 'vectors_Astronomy_Astrology_Cosmography.wv',
 'vectors_Biology.wv',
 'vectors_Geography_Cartography.wv',
 'vectors_Mathematics.wv',
 'vectors_Medicine.wv',
 'vectors_Meteorology_Earth_sciences.wv',
 'vectors_Physics.wv']

In [52]:
labels_vectors_tups = [('NOSCEMUS - 1501-1550', 'vectors_1501-1550.wv'),
 ('NOSCEMUS - 1551-1600', 'vectors_1551-1600.wv'),
 ('NOSCEMUS - 1601-1650', 'vectors_1601-1650.wv'),
 ('NOSCEMUS - 1651-1700', 'vectors_1651-1700.wv'),
 ('NOSCEMUS - Alchemy/Chemistry', 'vectors_Alchemy_Chemistry.wv'),
 ('NOSCEMUS - Astronomy/Astrology/Cosmography',
  'vectors_Astronomy_Astrology_Cosmography.wv'),
 ('NOSCEMUS - Biology', 'vectors_Biology.wv'),
 ('NOSCEMUS - Geography/Cartography', 'vectors_Geography_Cartography.wv'),
 ('NOSCEMUS - Mathematics', 'vectors_Mathematics.wv'),
 ('NOSCEMUS - Medicine', 'vectors_Medicine.wv'),
 ('NOSCEMUS - Meteorology/Earth sciences',
  'vectors_Meteorology_Earth_sciences.wv'),
 ('NOSCEMUS - Physics', 'vectors_Physics.wv')]


In [53]:
source_path = "/srv/data/tome/noscemus/vectors/"
vectors_dict_comp = {}
for tup in labels_vectors_tups:
    model = KeyedVectors.load(source_path + tup[1])
    word_vectors = model.vectors
    vocabulary = model.index_to_key
    kv = KeyedVectors(vector_size=word_vectors.shape[1])
    # Fill the keyed vectors
    for word, vector in zip(vocabulary, word_vectors):
        kv.add_vector(word, vector)
    vectors_dict_comp[tup[0]] = kv

In [54]:
vectors_dict_comp["NOSCEMUS - Biology"].vectors.shape

(5956, 100)

In [55]:
for label, model_path in [
        ("LASLA", "/srv/data/lila/allLASLA-lemmi-fast-100-SKIP-win5-min5.vec"),
        ("Opera Maiora" , "/srv/data/lila/opera-maiora-lemmas_skip_100.vec")]:
    model = KeyedVectors.load_word2vec_format(model_path, binary=False)
    word_vectors = model.vectors
    vocabulary = model.index_to_key
    kv = KeyedVectors(vector_size=word_vectors.shape[1])
    # Fill the keyed vectors
    for word, vector in zip(vocabulary, word_vectors):
        kv.add_vector(word, vector)
    vectors_dict_comp[label] = kv


In [56]:
with open("/srv/data/tome/noscemus/vectors/vectors_dict_comp.pkl", "wb") as file:
    pickle.dump(vectors_dict_comp, file)